In [1]:
!pip install wordcloud

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 371 kB 9.9 MB/s            
     |████████████████████████████████| 10.3 MB 23.1 MB/s            ████████▎              | 5.6 MB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 15.2 MB/s            
     |████████████████████████████████| 1.2 MB 27.1 MB/s            


In [3]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger

from wordcloud import WordCloud

from pymongo import MongoClient

import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'nltk'

### Connecting to database and getting texts

In [6]:
# Creating a connection to MongoDB
client = MongoClient("localhost", 27017)
db = client["news"]
collection = db["elespectador"]

NameError: name 'MongoClient' is not defined

In [ ]:
text = []
for news in list(collection.find({}, {"title": 1, "summary": 1, "full_text": 1, "_id": 0})):
    text.append(news["title"])
    text.append(news["summary"])
    text.append(news["full_text"])

In [ ]:
text = " ".join(text)

In [ ]:
text

### Lowercasing

In [ ]:
text = text.lower() # WARNING

### Tokenization

In [ ]:
sentences = sent_tokenize(text)

print(sentences)

In [ ]:
words = word_tokenize(text)

print(words)

### Words distribution

In [ ]:
frec_dist = FreqDist(words)

print(frec_dist)

In [ ]:
frec_dist.most_common(5)

In [ ]:
n = 10
plt.bar([ w[0] for w in frec_dist.most_common(n) ], [ w[1] for w in frec_dist.most_common(n) ])

In [ ]:
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(text)

plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

### Stopwords

Stop words are basically a set of commonly used words in any language, not just English.

The reason why stop words are critical to many applications is that, if we remove the words that are very commonly used in a given language, we can focus on the important words instead.

In [ ]:
stop_words = set(stopwords.words("spanish"))
print(stop_words)

In [ ]:
filtered_words = []
for w in words:
    if w not in stop_words:
        filtered_words.append(w)

print("All words:", words)
print("\n")
print("Substracting stopwords:", filtered_words)

In [ ]:
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(" ".join(filtered_words))

plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()

### Part of Speech

https://nlp.stanford.edu/software/

In [ ]:
jar = "./pos-tagger/stanford-postagger/stanford-postagger-4.2.0.jar"
model = "./pos-tagger/stanford-postagger/models/spanish-ud.tagger"

In [ ]:
java_path = "/usr/bin/java"
os.environ["JAVAHOME"] = java_path

In [ ]:
pos_tagger = StanfordPOSTagger(model, jar, encoding = "utf8")
pos_tagger.tag(words)